Database operations to build the structure for visualization

I will need 4 groups of data:
* topics (aggregator)
* docs
* words
* people

In [2]:
import os
import pandas as pd
import sqlite3
import json
import pickle
import re
import math
import gensim
from collections import Counter

In [30]:
inputs = os.path.join("..", "data")
topic_models = os.path.join("..", "data", "04_model")
outputs = os.path.join('..','data', '06_outputs')
tmp = os.path.join('..','data', 'tmp')

# get data source

In [25]:
sql_db = os.path.join(inputs, 'cpdoc_as.sqlite')
conn = sqlite3.connect(sql_db)
cur = conn.cursor()

topic_doc = pd.read_sql_query("SELECT * FROM topic_doc", conn) # loading from pkl below
docs = pd.read_sql_query("SELECT * FROM docs", conn)
persons = pd.read_sql_query("SELECT * FROM persons", conn)
person_doc = pd.read_sql_query("SELECT * FROM person_doc", conn)
topics_table = pd.read_sql_query("SELECT * FROM topics", conn)

# topics

In [26]:
topics_list = topics_table['old_id'].to_list()

# docs and topics

In [6]:
topic_doc = topic_doc.sort_values(['topic_id', 'topic_score'], ascending=[True, False])
print(len(topic_doc))
topic_doc.head()

114501


,doc_id,topic_id,topic_score,topic_id_renamed
38437,bp_1974.06.04_doc_I-8,0,0.802069,5
9794,ag_1974.01.22_doc_III-55,0,0.710054,5
100941,rb_1974.04.17_doc_I-25,0,0.695797,5
105727,rb_1974.05.23_doc_III-11,0,0.689144,5
105893,rb_1974.05.23_doc_III-22,0,0.685677,5


# docs and persons

In [7]:
# persons.rename(columns={"person_name": "name"}, inplace=True)
persons.rename(index=str, columns={"id": "person_id", "person_name": "name"}, inplace=True)

In [8]:
# person_doc = cpdoc_tables['person_doc ']
person_doc = pd.merge(person_doc, persons, on='person_id', how='inner')
person_doc = person_doc[['person_id', 'doc_id', 'person_count', 'name']]
person_doc = person_doc.replace('(.*),(.*),.*', r'\1,\2', regex=True)
person_doc = person_doc.replace('(.*), (.*)', r'\2 \1', regex=True)

In [9]:
#remove special characters
person_doc = person_doc.name.apply(pd.Series) \
    .replace('[“”]', '', regex=True) \
    .replace('[áàãâ]', 'a', regex=True) \
    .replace('[óòõô]', 'o', regex=True) \
    .replace('[éèê]', 'e', regex=True) \
    .replace('[íì]', 'i', regex=True) \
    .replace('[úù]', 'u', regex=True) \
    .replace('ç', 'c', regex=True) \
    .merge(person_doc, left_index = True, right_index = True) \
    .drop("name", axis=1) \
    .rename(columns={0: "name"})

In [10]:
person_doc.head()

,name,person_id,doc_id,person_count
0,Antonio Azeredo Da Silveira,500084,ag_1973.11.20_doc_I-8,1
1,Antonio Azeredo Da Silveira,500084,ag_1973.11.20_doc_I-9,1
2,Antonio Azeredo Da Silveira,500084,ag_1973.11.20_doc_I-13,1
3,Antonio Azeredo Da Silveira,500084,ag_1973.11.20_doc_I-22,1
4,Antonio Azeredo Da Silveira,500084,ag_1973.11.20_doc_I-23,1


In [11]:
person_doc_count = pd.DataFrame(person_doc.groupby(['name'])['person_count'].agg('count')) #.apply(sum)
person_doc_count = person_doc_count.reset_index()

# docs and tokens

In [12]:
docs.head()

,id,main_language,readability,url,body
0,ag_1973.11.20_doc_I-1,none,0.400000,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3..."
1,ag_1973.11.20_doc_I-4,none,-1.000000,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n""**; %wwffwç (. sz\nao?\n\nray\n\nmax meo..."
2,ag_1973.11.20_doc_I-5,none,-1.000000,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"am 1933 m 90\n\nw ij""?\n(\n\nt\n\ne\nzi\n\nurd..."
3,ag_1973.11.20_doc_I-7,pt,-1.000000,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"ôôô/ma 44 w\nwil""\n/ oe;\n\n \n\ncentro de es..."
4,ag_1973.11.20_doc_I-8,pt,0.615385,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nxx>4áiãfkk ;g*\n\nmv, ; (905\n\n \n\ntele..."


In [13]:
docs = docs[['id', 'url', 'body']]
docs = docs.rename(index=str, columns={"id": "doc_id"})
docs['length'] = docs['body'].apply(lambda x: len(x.split()))
docs.head()

,doc_id,url,body,length
0,ag_1973.11.20_doc_I-1,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3...",261
1,ag_1973.11.20_doc_I-4,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n""**; %wwffwç (. sz\nao?\n\nray\n\nmax meo...",54
2,ag_1973.11.20_doc_I-5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"am 1933 m 90\n\nw ij""?\n(\n\nt\n\ne\nzi\n\nurd...",64
3,ag_1973.11.20_doc_I-7,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"ôôô/ma 44 w\nwil""\n/ oe;\n\n \n\ncentro de es...",75
4,ag_1973.11.20_doc_I-8,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nxx>4áiãfkk ;g*\n\nmv, ; (905\n\n \n\ntele...",202


# words and topics

In [31]:
file_lda_model = os.path.join(topic_models,'model_lda_100_rs_00.pkl')
lda_model = pickle.load(open(file_lda_model, 'rb'))

In [33]:
def remove_special_char(text):
    text = re.sub('[áàãâ]', 'a', text)
    text = re.sub('[óòõô]', 'o', text)
    text = re.sub('[éèê]', 'e', text)
    text = re.sub('[íì]', 'i', text)
    text = re.sub('[úù]', 'u', text)
    text = re.sub('ç', 'c', text)
    return text

In [16]:
def retrieve_tokens(text, df):
    main_tokens = []
    for token in df['tokens']:
        if token in text:
            token = remove_special_char(token)
            main_tokens.append(token)
    return main_tokens

# build vis table

In [17]:
def build_vis_table(topic_id, renamed_id, df):
    topic_tokens = lda_model.print_topics(-1, num_words=20)[topic_id]
    
    tokens = []
    scores = []
    for i in topic_tokens[1].split('+'):
        token = re.sub('.*\*"(.*)".*', r'\1', i)
        score = re.sub(' *(.*)\*.*', r'\1', i)
        score = float(score)
        tokens.append(token)
        scores.append(score)
    token_score_dict = {'tokens': tokens, 'scores': scores}
    token_score = pd.DataFrame(token_score_dict)
    
    docs['tokens'] = docs['body'].apply(lambda text: retrieve_tokens(text, token_score))
    
    #filter by topic
    df = df.loc[df['topic_id'] == topic_id].sort_values(by=['topic_score'], ascending=False)
    df = df.head(20)
    
    ### merge topics and tokens
    df = pd.merge(df, docs, on='doc_id', how='inner')
    
    ### relate docs to list of persons
    #filters person_doc
    array = list(df['doc_id'])
    person_doc_filtered = person_doc.loc[person_doc['doc_id'].isin(array)]
    
    #apply list of persons
    person_doc_filtered = person_doc_filtered.groupby(['doc_id'])['name'].apply(list)
    person_doc_filtered = pd.DataFrame({'doc_id':person_doc_filtered.index, 'names':person_doc_filtered.values})
    
    #merge topics and persons
    df = pd.merge(df, person_doc_filtered, on='doc_id', how='outer')
#     df['year'] = df['date'].apply(lambda x: x.year)
    df = df.astype({'names': 'object'})
    for row in df.loc[df.names.isnull(), 'names'].index:
        df.at[row, 'names'] = []
#     df['topic_id'] = df['topic_id_renamed']
#     df.drop('topic_id_renamed', axis=1, inplace=True)
    df['topic_id'] = renamed_id
    
    return df

In [18]:
topic_vis_dict = {}

In [19]:
for i in range(100):
    renamed_id = topics_list.index(i)
    topic_vis_dict[renamed_id] = build_vis_table(i, renamed_id, topic_doc)

In [20]:
topic_vis_dict.keys()

dict_keys([5, 89, 58, 37, 43, 63, 12, 35, 49, 0, 84, 16, 64, 50, 68, 41, 40, 18, 54, 44, 85, 29, 57, 66, 73, 47, 76, 59, 4, 2, 92, 87, 3, 61, 42, 6, 94, 55, 24, 23, 52, 34, 17, 79, 7, 80, 65, 69, 93, 15, 81, 48, 46, 38, 8, 91, 19, 96, 56, 75, 95, 31, 51, 21, 74, 67, 78, 97, 45, 30, 98, 26, 88, 53, 90, 11, 86, 9, 62, 82, 25, 13, 10, 32, 1, 83, 27, 28, 60, 39, 72, 20, 22, 77, 33, 36, 70, 71, 99, 14])

7=35, 49=15

In [22]:
topic_vis_dict[35].tail()

,doc_id,topic_id,topic_score,topic_id_renamed,url,body,length,tokens,names
15,pn_1975.00.00_doc_6,35,0.310558,35,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n \n\ná4ó (%?ç'quoicg; vê!/avó m & ""l...",251,"[nuclear, nucleares, brasil, armas, tratado]",[]
16,pn_1974.08.15_doc_III-31,35,0.303900,35,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[nuclear, acordo, energia, nucleares, brasil, ...","[Jimmy Carter, Cyrus Vance, Helmut Schmidt]"
17,pn_1976.12.28_doc_29,35,0.300889,35,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[nuclear, acordo, energia, nucleares, brasil, ...",[Jimmy Carter]
18,pn_1974.08.15_doc_II-1,35,0.290115,35,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, acordo, energia, nucleares, brasil, ...",[Jimmy Carter]
19,pn_1976.12.28_doc_16,35,0.286767,35,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[acordo, energia, brasil, uranio, rfa, salvagu...",[Antonio Azeredo Da Silveira]


In [23]:
topic_vis_dict[15].tail()

,doc_id,topic_id,topic_score,topic_id_renamed,url,body,length,tokens,names
15,bp_1977.03.10_doc_V-3,15,0.471688,15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,". ""* aas_ manoa-to\n/ . _ - . . awm)\n\nsala-1...",6339,"[itaipu, rio, cota, operacao, energia, constru...","[Antonio Azeredo Da Silveira, Shigeaki Ueki, C..."
16,bp_1977.09.13_doc_II-2,15,0.469913,15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,- informação no? 332/76 - estudo sobre a fixaç...,2444,"[itaipu, rio, corpus, cota, operacao, energia,...",[]
17,d_1974.03.26_doc_XXXI-44,15,0.466800,15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n\n \n\nwm\n""mg . .\na"" rio âecreto\n...",6152,"[itaipu, rio, cota, operacao, energia, constru...","[Antonio Azeredo Da Silveira, Costa Cavalcanti..."
18,bp_1977.09.13_doc_I-14,15,0.455082,15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"/\n\n \n\n \n\n \n\na\n\n \n\n!\n\n \n\n o:""i...",1690,"[itaipu, rio, corpus, cota, construcao, parana...",[]
19,d_1974.04.23_doc_XXXII-8,15,0.454966,15,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nwuxa\n\naas, ao34as3'13\nqm\n\n secreto\n...",48,"[itaipu, rio, barragem, iguacu, eletrobras]",[]


# pickle file

In [7]:
topics_dict_file = os.path.join(tmp, '06_topics_dict.pkl')
if os.path.isfile(topics_dict_file): 
    user_input = input("file already exists on folder. Do you want to replace it or do you want to use it? "
                       "Type 'replace' or 'use' on your keyboard: ")
    if user_input.lower() == 'replace':
        print('\r\nReplacing data.')
        with open(topics_dict_file, 'wb') as f:
            pickle.dump(topic_vis_dict, f)
    else: 
        print('\r\nUsing preexisting data.')
        with open(topics_dict_file, 'rb') as f:
            topic_vis_dict = pickle.load(f)
else:
with open(topics_dict_file, 'wb') as f:
    pickle.dump(topic_vis_dict, f)    

file already exists on folder. Do you want to replace it or do you want to use it? Type 'replace' or 'use' on your keyboard: d

Using preexisting data.


In [27]:
topics = range(100)

# Visualization of a specific topic

list split  in pandas: https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401

In [8]:
nuclear_brazil_df = topic_vis_dict[35]
nuclear_brazil_df['doc'] = nuclear_brazil_df['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
cols = nuclear_brazil_df.columns.tolist()
cols = cols[:3] + cols[4:] + cols[3:4]
nuclear_brazil_df = nuclear_brazil_df[cols]
nuclear_brazil_df.tail()

,doc_id,topic_id,topic_score,url,body,length,tokens,names,doc,topic_id_renamed
15,pn_1975.00.00_doc_6,35,0.310558,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n \n\ná4ó (%?ç'quoicg; vê!/avó m & ""l...",251,"[nuclear, nucleares, brasil, armas, tratado]",[],doc_6,35
16,pn_1974.08.15_doc_III-31,35,0.303900,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,secreto-excâusãvo\n\nsubsídios para as consult...,4549,"[nuclear, acordo, energia, nucleares, brasil, ...","[Jimmy Carter, Cyrus Vance, Helmut Schmidt]",doc_III-31,35
17,pn_1976.12.28_doc_29,35,0.300889,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"telegrama recebido '\n\naaaa [97034 . é?\no"" '...",845,"[nuclear, acordo, energia, nucleares, brasil, ...",[Jimmy Carter],doc_29,35
18,pn_1974.08.15_doc_II-1,35,0.290115,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\nmmm\n\n""a. . .. , . ..\n\n' o acordo nucl...",1672,"[nuclear, acordo, energia, nucleares, brasil, ...",[Jimmy Carter],doc_II-1,35
19,pn_1976.12.28_doc_16,35,0.286767,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,antonio la'. aiii-1315230 da sxmrâxm\njáwówa i...,330,"[acordo, energia, brasil, uranio, rfa, salvagu...",[Antonio Azeredo Da Silveira],doc_16,35


# build main data for each topic

In [27]:
topics = range(100)

In [13]:
topic_vis_dict[5].head()

,doc_id,topic_id,topic_score,topic_id_renamed,url,body,length,tokens,names,doc
0,bp_1974.06.04_doc_I-8,5,0.802069,5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,aas _ aaaa. 05.027\nmxukbp\n\n secreto em 18 d...,149,"[excelencia, senhor, exteriores, relacoes, min...",[Antonio Azeredo Da Silveira],doc_I-8
1,ag_1974.01.22_doc_III-55,5,0.710054,5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"atx (qqá ç(, ?l\n\n(n*-* , v , )""\n\naviso-cir...",255,"[excelencia, senhor, exteriores, relacoes, min...",[Antonio Azeredo Da Silveira],doc_III-55
2,rb_1974.04.17_doc_I-25,5,0.695797,5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n' as-'4ªl""+u__o<_i_\n. w_, _i__:22?$?u, ,...",712,"[excelencia, senhor, exteriores, relacoes, min...",[Antonio Azeredo Da Silveira],doc_I-25
3,rb_1974.05.23_doc_III-11,5,0.689144,5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nav. n? 173\nem 04 de março de 1975\n...,264,"[excelencia, senhor, exteriores, relacoes, min...","[Antonio Azeredo Da Silveira, Golbery Do Couto...",doc_III-11
4,rb_1974.05.23_doc_III-22,5,0.685677,5,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n \n\n"" w kal? 'é &]hãú _ , l "" . ' lyx ta...",263,"[excelencia, senhor, exteriores, relacoes, min...",[Antonio Azeredo Da Silveira],doc_III-22


In [17]:
dict_topics_json = {}
for topic in topics:
    #rename docs
    dict_topics_json[topic] = topic_vis_dict[topic]
    dict_topics_json[topic]['doc'] = dict_topics_json[topic]['doc_id'].apply(lambda text: re.sub('.*(doc.*)', r'\1', text))
    
    #places date column at the end of df
    cols = dict_topics_json[topic].columns.tolist()
    cols = cols[:3] + cols[4:] + cols[3:4] 
    dict_topics_json[topic] = dict_topics_json[topic][cols]
    
    topic_transposed = dict_topics_json[topic] \
        .drop(["body", "topic_id"], axis = 1) \
        .transpose()
#         .rename(columns={"pdf": "url"}) \

    
    # build json of main data
    topic_json = topic_transposed.to_json(orient='columns')
    topic_json = re.sub('"\d+":', '', topic_json)
    topic_json = re.sub("^{", "[", topic_json)
    topic_json = re.sub("}$", "]", topic_json)
    topic_json = json.loads(topic_json)
    
    # save json
    json_file = 'topic_{}.json'.format(str(topic))
    file_path = os.path.join(outputs, json_file)
    json.dump(topic_json, open(file_path, 'w'))

# define function to melt dataframes

In [18]:
def get_melted_df(df,variable):
    cols_to_drop = set(df.columns) - set(['doc_id'])
    topic = df[variable].apply(pd.Series) \
        .merge(df, left_index = True, right_index = True) \
        .drop(cols_to_drop, axis = 1) \
        .melt(id_vars = ['doc_id'], value_name = variable) \
        .drop("variable", axis = 1) \
        .dropna()
    return topic

# generate json files related to persons
Get connections between docs and persons. Those represent the edges of a graph.

In [19]:
b, a = 2, 1
count_min, count_max = 0, 20 #len(docs)
for topic in topics:
    #build list of docs for each person
    melted_df = get_melted_df(topic_vis_dict[topic], 'names')
    melted_df = melted_df.groupby(['names'])['doc_id'].apply(list)
    melted_df = pd.DataFrame({'name':melted_df.index, 'docs':melted_df.values})
    
    # build json of list of names
    names_list = []
    for index, row in melted_df.iterrows():
        name = row['name']
        row_docs = row['docs']
        name_count = len(row_docs)
        
        temp_dict = {'name': name, 'count': name_count, 'docs': row_docs}
        names_list.append(temp_dict)
        
    #feature scaling to a set of values
    count_list = [i['count'] for i in names_list]
    for i in range(len(names_list)):
        names_list[i]['count'] = (names_list[i]['count']-count_min)/(count_max-count_min)
        
    # save json
    file_path = os.path.join(outputs,'names_list_{}.json'.format(str(topic)))
    json.dump(names_list, open(file_path, 'w'))

# generate json files related to tokens
Get connections between docs and tokens. Those represent the edges of a graph.

In [34]:
for topic in topics:
    #build list of docs for each token
    melted_df = get_melted_df(topic_vis_dict[topic], 'tokens')
    melted_df = melted_df.groupby(['tokens'])['doc_id'].apply(list)
    melted_df = pd.DataFrame({'token':melted_df.index, 'docs':melted_df.values})
    
    #get data from lda_model
    old_topic = topics_list[topic]
    topic_tokens = lda_model.print_topics(-1, num_words=20)[old_topic]
    topic_tokens = topic_tokens[1].split('+')
    
    #topic_tokens will be a list with elements such as: '0.085*"nuclear" ',
    tokens_list = []
    for pair in topic_tokens:
        #get tokens and scores
        token = re.sub('.*\*"(.*)".*', r'\1', pair)
        token = remove_special_char(token)
        score = re.sub(' *(.*)\*.*', r'\1', pair)
        score = float(score)

        #get list of docs
        docs = melted_df[melted_df['token'] == token]['docs'].tolist()
        if docs: 
            docs = docs[0]
            temp_dict = {'token': token, 'score': score, 'docs': docs}
            tokens_list.append(temp_dict)
    file_path = os.path.join(outputs,'tokens_list_{}.json'.format(str(topic)))
    json.dump(tokens_list, open(file_path, 'w'))

# build object for d3 observable vis
To build object, run d3_objects_str_builder.ipynb